In [1]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Masking,Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed,MaxPooling1D, Flatten, Conv1D,Conv2D,Dropout, MaxPooling2D # for creating layers inside the Neural Network
from keras.optimizers import Adam , SGD
# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling
from sklearn.model_selection import train_test_split
# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version
from matplotlib import pyplot as plt

#file accessing
import os
# time stuff
from datetime import timedelta
import calendar


c:\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


Tensorflow/Keras: 2.9.0
pandas: 1.4.2
numpy: 1.22.4
sklearn: 1.1.1
plotly: 5.9.0


In [2]:
#@title Define the plotting functions
def plot_the_model(trained_weight, trained_bias, feature, label):
  """Plot the trained model against the training feature and label."""

  # Label the axes.
  plt.xlabel("feature")
  plt.ylabel("label")

  # Plot the feature values vs. label values.
  plt.scatter(feature, label)

  # Create a red line representing the model. The red line starts
  # at coordinates (x0, y0) and ends at coordinates (x1, y1).
  x0 = 0
  y0 = trained_bias
  x1 = feature[-1]
  y1 = trained_bias + (trained_weight * x1)
  plt.plot([x0, x1], [y0, y1], c='r')

  # Render the scatter plot and the red line.
  plt.show()

def plot_the_loss_curve(epochs, rmse):
  """Plot the loss curve, which shows loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")
  #epochs.remove(max(epochs))
  
  #rmse.drop(max(rmse))
  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.97, rmse.max()])
  plt.show()

print("Defined the plot_the_model and plot_the_loss_curve functions.")

Defined the plot_the_model and plot_the_loss_curve functions.


##### Read from file and establish dataframe and future functions

In [3]:
df=pd.read_csv('E:/DSFellowship/InSAR_data_south/displacement/export_dataframe1.csv')
df=df.set_index([df.columns[0],df.columns[1]])
df.columns=pd.to_datetime(df.columns, format='%Y%m%d')
df=df.dropna(axis=0, how='all')#drop full nan rows

##### Remove next line to include all data

In [4]:
df=df.iloc[100000:500000] #cuts data to long 120-119 approx
df

2014-11-08  2014-12-02  2014-12-26  2015-02-12  \
Longitude Latitude                                                   
-120.5647 36.5493          0.0    -7.36750     0.30839     0.80180   
          36.5513          0.0    -7.37000     0.31268     0.82006   
          36.5533          0.0    -7.35920     0.30634     0.82830   
          36.5553          0.0    -7.36650     0.35908     0.90634   
          36.5573          0.0    -7.38350     0.32145     0.88016   
...                        ...         ...         ...         ...   
-119.9868 37.4516          0.0    -0.89138    -2.25990     1.94810   
          37.4536          0.0    -0.94919    -2.26130     1.95250   
          37.4576          0.0    -1.00920    -2.37100     1.80360   
          37.4596          0.0    -1.08340    -2.45840     1.76390   
          37.4616          0.0    -1.18290    -2.57200     1.66300   

                    2015-03-08  2015-04-01  2015-04-25  2015-05-19  \
Longitude Latitude                                                   
-120.5647 36.5493     -0.63115      2.5428   -0.243660    3.414000   
          36.5513     -0.64685      2.6231   -0.262060    3.447800   
          36.5533     -0.63582      2.6812   -0.238590    3.457800   
          36.5553     -0.54705      2.8056   -0.183480    3.521600   
          36.5573     -0.57876      2.8102   -0.215540    3.464300   
...                        ...         ...         ...         ...   
-119.9868 37.4516     -3.06210     -2.8175   -0.430170    0.017711   
          37.4536     -3.07290     -2.8799   -0.409550   -0.138200   
          37.4576     -3.21370     -2.9556    0.210320   -0.210810   
          37.4596     -3.30260     -2.8998   -0.095809   -0.074748   
          37.4616     -3.42820     -3.0511   -0.617990   -0.166100   

                    2015-06-12  2015-07-06  ...  2018-10-18  2018-10-30  \
Longitude Latitude                          ...                           
-120.5647 36.5493      5.44400    2.463200  ...     1.20690    4.326300   
          36.5513      5.43260    2.462700  ...     1.32300    4.448600   
          36.5533      5.43910    2.459600  ...     1.07790    4.209400   
          36.5553      5.50390    2.492000  ...     1.06510    4.210700   
          36.5573      5.48210    2.440500  ...     0.87105    4.022100   
...                        ...         ...  ...         ...         ...   
-119.9868 37.4516      0.33933    0.212400  ...    -2.20070   -0.226000   
          37.4536      0.14295    0.018685  ...    -2.84590   -0.835820   
          37.4576      0.20266    0.081038  ...    -2.15880   -0.071612   
          37.4596      0.40247    0.275190  ...    -2.80260   -0.650980   
          37.4616      0.36765    0.247280  ...    -2.87630   -0.670420   

                    2018-11-11  2018-11-17  2018-11-23  2018-12-05  \
Longitude Latitude                                                   
-120.5647 36.5493      3.58960     3.48600    0.532130    0.047884   
          36.5513      3.72920     3.61550    0.579940    0.109060   
          36.5533      3.50080     3.37520    0.287090   -0.170190   
          36.5553      3.51870     3.38010    0.233830   -0.222880   
          36.5573      3.33110     3.19180   -0.020346   -0.439840   
...                        ...         ...         ...         ...   
-119.9868 37.4516     -0.58880    -0.17788   -0.584070   -2.059900   
          37.4536     -1.20170    -0.82906   -1.096100   -2.607900   
          37.4576     -0.44757    -0.11933   -0.324470   -2.064700   
          37.4596     -1.05400    -0.75891   -0.833980   -2.781700   
          37.4616     -1.11450    -0.85133   -0.872230   -2.896600   

                    2018-12-17  2018-12-29  2019-01-10  2019-01-22  
Longitude Latitude                                                  
-120.5647 36.5493       2.6626      3.7143    3.947300     5.44600  
          36.5513       2.7313      3.7821    4.003100     5.52030  
          36.5533       2.4374     

# Timestep and features

In [5]:
timestep=10
features=1

# Shaping function (remove start and step for first for loop to include all data)

In [6]:
def shaping(datain, timestep):
    #print(arr)
    cnt=0
    
    for row in range(0,len(datain.index),500): #picks a row at every iteration, allows to reduction of input and inclusion of multiple time series, remove start and step to run on full dataset 
    # Convert input dataframe to array and flatten
        arr=datain.iloc[row].to_numpy().flatten() # flatten row
        arr[np.isnan(arr)] = -1000
        for mth in range(0, len(datain.columns)-(2*timestep)+1): # Define range lenght of the dates - 2* amount of timesep?? +1
            cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
            X_start=mth # Start month for inputs of each sample
            X_end=mth+timestep # End month for inputs of each sample
            Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
            Y_end=mth+2*timestep # End month for targets of each sample.  
            
            # Assemble input and target arrays containing all samples
            if cnt==1:
                X_comb=arr[X_start:X_end]
                Y_comb=arr[Y_start:Y_end]
            else: 
                X_comb=np.append(X_comb, arr[X_start:X_end])
                Y_comb=np.append(Y_comb, arr[Y_start:Y_end])
    
    # Reshape input and target arrays
    X_out=np.reshape(X_comb, (cnt, timestep, 1))
    Y_out=np.reshape(Y_comb, (cnt, timestep, 1))
    return X_out, Y_out

##### Step 2 - Split and Shape data

In [7]:
#Divides data into 80% training 20% testing
train, test = train_test_split(df, test_size=0.2)
# creates sequences to train
X_train, Y_train = shaping(datain=train, timestep= timestep )
X_test, Y_test = shaping(datain=test, timestep=timestep)

##### Step 3 - Specify the structure of a Neural Network, first model

In [8]:
model2 = Sequential(name="LSTM-Model") # Model
model2.add(Masking(mask_value=-1000, input_shape=(timestep, 1)))
model2.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model2.add(Bidirectional(LSTM(units=64, activation='relu', recurrent_activation='sigmoid', stateful=False), name='Hidden-LSTM-Encoder-Layer')) # Encoder Layer
model2.add(RepeatVector(Y_train.shape[1], name='Repeat-Vector-Layer')) # Repeat Vector
model2.add(Bidirectional(LSTM(units=64, activation='relu', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-LSTM-Decoder-Layer')) # Decoder Layer
model2.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x
#optimizer=Adam(.001)
model2.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
        
             )

In [9]:
##### Train model

In [10]:

history = model2.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=3, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=400, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=2, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
                    #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=100, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
                
                   )




Epoch 1/400
15531/15531 - 226s - loss: 9096.6514 - mean_squared_error: 9096.5918 - mean_absolute_error: 17.7718 - 226s/epoch - 15ms/step
Epoch 2/400
15531/15531 - 218s - loss: 8480.3057 - mean_squared_error: 8480.2930 - mean_absolute_error: 17.8115 - 218s/epoch - 14ms/step
Epoch 3/400
15531/15531 - 220s - loss: 7675.9087 - mean_squared_error: 7675.9204 - mean_absolute_error: 17.6410 - 220s/epoch - 14ms/step
Epoch 4/400
15531/15531 - 220s - loss: 6991.5288 - mean_squared_error: 6991.5117 - mean_absolute_error: 16.4788 - 220s/epoch - 14ms/step
Epoch 5/400
15531/15531 - 219s - loss: 5585.7285 - mean_squared_error: 5585.7227 - mean_absolute_error: 14.3952 - 219s/epoch - 14ms/step
Epoch 6/400
15531/15531 - 222s - loss: 5291.9521 - mean_squared_error: 5291.9424 - mean_absolute_error: 13.8994 - 222s/epoch - 14ms/step
Epoch 7/400
15531/15531 - 226s - loss: 4854.2021 - mean_squared_error: 4854.2056 - mean_absolute_error: 13.0498 - 226s/epoch - 15ms/step
Epoch 8/400
15531/15531 - 217s - loss: 42

##### Step 4 - Plot loss over epochs

In [ ]:
# Gather the trained model's weight and bias.
trained_weight = model2.get_weights()[0]
trained_bias = model2.get_weights()[1]

  # The list of epochs is stored separately from the 
  # rest of history.
epochs = history.epoch
  
  # Gather the history (a snapshot) of each epoch.
hist = pd.DataFrame(history.history)

  # Specifically gather the model's root mean 
  # squared error at each epoch. 
rmse = hist["mean_squared_error"]

#plot_the_model(trained_weight, trained_bias, X_test, Y_train)
plot_the_loss_curve(epochs, rmse)

##### Step 5 - Print Performance Summary

In [ ]:
print("")
print('-------------------- Model Summary --------------------')
model2.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Too many parameters to print but you can use the code provided if needed")
print("")
#for layer in model.layers:
#    print(layer.name)
#    for item in layer.get_weights():
#        print("  ", item)
#print("")

# Print the last value in the evaluation metrics contained within history file
print('-------------------- Evaluation on Training Data --------------------')
for item in history.history:
    print("Final", item, ":", history.history[item][-1])
print("")

# Evaluate the model on the test data using "evaluate"
print('-------------------- Evaluation on Test Data --------------------')
results = model2.evaluate(X_test, Y_test)
print("")